## Sentiment Analysis.py 
----
The purpose of this code is to analyze the sentiment of any tweeter user input into this file to then correlate it bitcoin price. The goal is to be able to find users who can relatively predict the price of bitcoin up to a few days later.

Libraries will include, numpy, pandas, matplotlib, tweepy, seaborn, and plotly.



In [130]:
# Dependencies
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import tweepy
import time
import seaborn as sns

# Initialize Sentiment Analyzer 
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [131]:
# Twitter API Keys

#default key
consumer_key = "jCuMds8hkjry8JV8JDEuDVH9o"
consumer_secret = "psgKB7nb05kZqoD2ZFPrG78OqbObHySWUEhcLFcZ03qVMlsCwp"
access_token = "814999527451148288-PVho6BBmmcQbSVKOHBt3E5jbPJM6Krl"
access_token_secret = "a30jMaE70P2kefPFOzrfGTlA06okUcifkjJB9g2JWq4Ih"

#mykey
#consumer_key = "NqScljb6fcL4WHIuN9iHgrkj2"
#consumer_secret = "bcCAGuzpSIfjWFDQfj6FbqabUN1ReIXBHwpmuKJlK2kthuuPYr"
#access_token = "894209492157554693-d4Lb74DTmG7DY4JYPAbfLL1RzGsWFfR"
#access_token_secret = "9IUVfRYP76fWRaNPP9elgsTyIq01gPpmw1HH0NOSk5tz8"

In [132]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

## Enter Twitter handle below

In [133]:
user_input = input("Enter the name of a twitter handle: @") 

Enter the name of a twitter handle: @gavinandresen


In [134]:
news_source = [user_input]


In [135]:
num_tweets = input("How many tweets? ") 
num_tweets = int(num_tweets) / 20
num_tweets = int(num_tweets)
num_tweets = range(num_tweets)

How many tweets? 300


In [136]:
# Override user input TEMPORARILY for code to work
# Pending to create if user error does not exist

#news_source = ["ErikVoorhees"]
#news_source

# Other sources tried: TuurDemeester, ErikVoorhees

## Creates function to grab the last x tweets from specified twitter handle

In [137]:
# Create a generic dictionary for holding all tweet information
tweet_data = {
    "tweet_source": [],
    "tweet_text": [],
    "tweet_date": [],
    "tweet_vader_score": [],
    "tweet_neg_score": [],
    "tweet_pos_score": [],
    "tweet_neu_score": []
}

# Grab 20 tweets from each page (total 400)

#Each page 

# original code for below line: "for x in range(30)"

for x in num_tweets:

    # Loop through all news sources
    for source in news_source:

        # Grab the tweets
        tweets = api.user_timeline(source, page=x)

        # For each tweet store it into the dictionary
        for tweet in tweets:
            
            # All data is grabbed from the JSON returned by Twitter
            tweet_data["tweet_source"].append(tweet["user"]["name"])
            tweet_data["tweet_text"].append(tweet["text"])
            tweet_data["tweet_date"].append(tweet["created_at"])

            # Run sentiment analysis on each tweet using Vader
            tweet_data["tweet_vader_score"].append(analyzer.polarity_scores(tweet["text"])["compound"])
            tweet_data["tweet_pos_score"].append(analyzer.polarity_scores(tweet["text"])["pos"])
            tweet_data["tweet_neu_score"].append(analyzer.polarity_scores(tweet["text"])["neu"])
            tweet_data["tweet_neg_score"].append(analyzer.polarity_scores(tweet["text"])["neg"])

In [138]:
# Store the final contents into a DataFrame
tweet_df = pd.DataFrame(tweet_data, columns=["tweet_source", 
                                             "tweet_text", 
                                             "tweet_date",
                                             "tweet_vader_score",
                                             "tweet_pos_score",
                                             "tweet_neu_score",
                                             "tweet_neg_score"])

# Visualize the DataFrame
#tweet_df.tail(3)

In [139]:
# Count the total number of tweets
# tweet_df.count()

## Initial cleaning of data

In [140]:
tweet_df['tweet_date'] = pd.to_datetime(tweet_df['tweet_date'])
tweet_df['tweet_date'] = tweet_df['tweet_date'].dt.strftime('%Y/%m/%d')
tweet_df.reset_index(drop=True, inplace=True)
# tweet_df

In [141]:
# tweet_df['tweet_source'][0]

## Sum and plot data of specified user

In [142]:
import datetime as dt
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

In [143]:
plot_df = tweet_df.loc[tweet_df['tweet_source'] == tweet_df['tweet_source'][0]]
plot_df = plot_df[["tweet_source", "tweet_date", "tweet_vader_score", "tweet_text"]]
plot_df = plot_df.groupby(['tweet_source','tweet_date']).mean()
plot_df.reset_index(drop=False, inplace=True)
plot_df.sort_values('tweet_date', inplace=True)
plot_df.reset_index(drop=True, inplace=True)
plot_df = plot_df.rename(columns={'tweet_vader_score': 'mean_tweet_vader_score'})
plot_df['two_day_mov_avg'] = plot_df['mean_tweet_vader_score'].rolling(window=3).mean()
plot_df['seven_day_mov_avg'] = plot_df['mean_tweet_vader_score'].rolling(window=7).mean()

plot_df = plot_df.rename(columns={'tweet_date': 'Date'})

# plot_df.tail(10)


#print(" --------------------------------------------------------------------------")
#print("|                                                                          |")
#print("|  Sentiment Analysis of " + tweet_df['tweet_source'][0] + " from  " + plot_df['tweet_date'][0] + "  to  " +  plot_df["tweet_date"].iloc[-1] + "   |")
#print("|                                                                          |")
#print(" --------------------------------------------------------------------------")

In [144]:
file_name = str(tweet_df['tweet_source'][0]) + "-" + str(len(plot_df)) +"-tweets.csv"

# tweet_df.to_csv("analysis/" + file_name, encoding="utf-8")

In [145]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [146]:
plot_1 = Scatter(x = plot_df['Date'], y = plot_df['mean_tweet_vader_score'])
plot_2 = Scatter(x = plot_df['Date'], y = plot_df['two_day_mov_avg'])
plot_3 = Scatter(x = plot_df['Date'], y = plot_df['seven_day_mov_avg'])

data = Data([plot_1, plot_2, plot_3])

layout = Layout(
    showlegend=True
)

fig = dict( data=data, layout=layout )

iplot(fig)

#save CSV
csv_name = tweet_df['tweet_source'][0]

#plot_1.to_csv(csv_name)

## Getting Bitcoin price history

In [147]:
import pandas as pd
import numpy as np
#import pandas_profiling

import seaborn as sns
import matplotlib.pyplot as plt
get_ipython().magic('matplotlib inline')
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True) 


import time
import matplotlib.dates as mdates
from datetime import datetime


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

coin = input("What coin do you want to look up? ")

# get market info for bitcoin from the start of 2013 to the current day
bitcoin_df = pd.read_html("https://coinmarketcap.com/currencies/" + coin + "/historical-data/?start=20170901&end="+time.strftime("%Y%m%d"))[0]

# convert the date string to the correct date format
bitcoin_df = bitcoin_df.assign(Date=pd.to_datetime(bitcoin_df['Date']))

# when Volume is equal to '-' convert it to 0
#bitcoin_df.loc[bitcoin_df['Volume']=="-",'Volume']=0

# convert to int
bitcoin_df['Volume'] = bitcoin_df['Volume'].astype('int64')

# sort values from most recent to January 2017
bitcoin_df = bitcoin_df.sort_values('Date',ascending=True)

#change to same datetime as twitter df
bitcoin_df['Date'] = bitcoin_df['Date'].dt.strftime('%Y/%m/%d')


What coin do you want to look up? bitcoin


In [148]:
bitcoin_2017 = bitcoin_df[(bitcoin_df['Date']>'2017-01-01')]

bitcoin_2017 = bitcoin_2017[["Date","Open","Close"]]
bitcoin_2017['Change'] = bitcoin_2017['Close'] - bitcoin_2017['Open']
# bitcoin_2017.head()

# For Up or Down in Price
change_dummy = []
for i, row in bitcoin_2017.iterrows():
    if row.Change >= 0:
        pos = 1
        change_dummy.append(pos)
        
    else:
        neg = 0
        change_dummy.append(neg)

bitcoin_2017['Change_dummy'] = change_dummy
# bitcoin_2017.head()

In [149]:
import plotly.plotly as py
import plotly.graph_objs as go

bit_plot = go.Scatter(x = bitcoin_2017['Date'], y=bitcoin_2017['Close'], name='Bitcoin History')

data_0 = Data([bit_plot])

layout_0 = go.Layout(
    title="Predicting Bitcoin Price with Specific Tweeter User's Sentiment",
    yaxis=dict(
        title='Bitcoin price',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)
fig = go.Figure(data=data_0, layout=layout_0)
plot_url = py.plot(fig, filename='multiple-axes-double')

fig = dict( data=data_0, layout=layout_0 )

iplot(fig)

Aw, snap! We don't have an account for ''. Want to try again? You can authenticate with your email address or username. Sign in is not case sensitive.

Don't have an account? plot.ly

Questions? accounts@plot.ly


## Combining bitcoin_df with tweet_df

In [150]:
comb_df = pd.merge(plot_df, bitcoin_2017, on='Date')
# comb_df.head(10)

In [151]:
comb_df = comb_df.iloc[7:]
# comb_df.head(3)

## Visualizing data

In [152]:
import plotly.plotly as py
import plotly.graph_objs as go


plot_1 = Scatter(x = comb_df['Date'], y = comb_df['mean_tweet_vader_score'], name='Average Day Score')
plot_2 = Scatter(x = comb_df['Date'], y = comb_df['two_day_mov_avg'], name='Two Day Moving Average')
plot_3 = Scatter(x = comb_df['Date'], y = comb_df['seven_day_mov_avg'], name='Seven Day Moving Average')
second_plot = go.Scatter(x = comb_df['Date'], y=comb_df['Close'], name='Bitcoin History', yaxis='y2')

data = Data([plot_1, plot_2, plot_3, second_plot])

layout = go.Layout(
    title="Predicting Bitcoin Price with Specific Tweeter User's Sentiment",
    yaxis=dict(
        range=[-1, 1],
        title='Sentiment Analysis'
    ),
    yaxis2=dict(
        title='Bitcoin price',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url = py.plot(fig, filename='multiple-axes-double')

fig = dict( data=data, layout=layout )

iplot(fig)

#save CSV
#csv_name = tweet_df['tweet_source'][0]

Aw, snap! We don't have an account for ''. Want to try again? You can authenticate with your email address or username. Sign in is not case sensitive.

Don't have an account? plot.ly

Questions? accounts@plot.ly
